In [13]:
import pypsa

In [61]:
network = pypsa.Network()

# Create an empty network.
network.set_snapshots(range(4))

# Add a bus to the network.
network.add("Bus","bus")

# Add a coal power plant to the bus.
network.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       p_nom=10000)

# Add a gas power plant to the bus.
network.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=25,
       p_min_pu=0.1,
       p_nom=1000)

# Add load to the bus.
network.add("Load","load",bus="bus",p_set=[4000,6000,5000,800])

In [62]:
# Solve the network.
network.lopf(network.snapshots,solver_name='gurobi')

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x21
  Lower bound: 320000.0
  Upper bound: 320000.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 21
  Number of binary variables: 8
  Number of integer variables: 8
  Number of continuous variables: 13
  Number of nonzeros: 45
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

('ok', 'optimal')

In [63]:
# Print the dispatch of each generator.
network.generators_t.p

,coal,gas
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


## Below here is MGA stuff


In [64]:
import pyomo.environ as pyomo_env

# Saving the value of the old objective function.
old_objective_value = network.model.objective()
print(old_objective_value)



320000.0


In [65]:
def extra_functionality(network, snapshots,  MGA_slack = 0.5):

    # Identify the nonzero decision variables that should enter the MGA objective function.
    # NB this line is not finished as all variables are selected now.
    #nonzero_gen_p =list(gen_p for gen_p in network.model.generator_p)
    nonzero_gen_p =list(gen_p for gen_p in network.model.generator_p if any(r in gen_p for r in ['coal']))
    
    # Build new MGA objective function.
    MGA_objective = 0
    for gen_p in nonzero_gen_p:
        MGA_objective += network.model.generator_p[gen_p]

    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=MGA_objective)

    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()

    # Add the MGA slack constraint.
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)

In [73]:
# Solve the network.
MGA_slack = 0.01
network.lopf(network.snapshots,solver_name='gurobi',extra_functionality=lambda network, snapshots: extra_functionality(network, snapshots, MGA_slack))
    

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x21
  Lower bound: 14360.0
  Upper bound: 14360.0
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 21
  Number of binary variables: 8
  Number of integer variables: 8
  Number of continuous variables: 13
  Number of nonzeros: 53
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (s

('ok', 'optimal')

In [74]:
# Print the dispatch of each generator.
network.generators_t.p

,coal,gas
0,4000.0,0.0
1,6000.0,0.0
2,4360.0,640.0
3,0.0,800.0


In [75]:
nonzero_gen_p =list(gen_p for gen_p in network.model.generator_p if any(r in gen_p for r in ['coal']))
nonzero_gen_p

[('coal', 0), ('coal', 1), ('coal', 2), ('coal', 3)]

In [76]:
network.model.objective()

323200.0